# Download the roads data in the hexagrid

Question: How many m/km of roads were mapped with RapID in selected region?

#### Import libs

In [1]:
import json
import os
import requests
import glob
from tqdm import tqdm
import time

### Get the data for every specified region

#### Definitions

In [2]:
os.chdir('..')


In [3]:
def get_geojson_names():

    home_dir = os.getcwd()
    geojson_dir = os.path.join(home_dir, f"geojson-regions\\geojson-hexagrid-states", "")

    # Construct the file pattern
    file_pattern = os.path.join(geojson_dir, '*.geojson')
    # print(file_pattern)

    # Use glob to get the list of file names matching the pattern
    file_names = glob.glob(file_pattern)
    # print(file_names)

    # Extract the base names of the files without the extension
    names = [os.path.splitext(os.path.basename(file_name))[0]
             for file_name in file_names]
    # print(names)
    # Print the names

    return names


geojson_names = get_geojson_names()
geojson_names


['geojson-hexagrid-india',
 'geojson-hexagrid-kenya',
 'geojson-hexagrid-nigeria',
 'geojson-hexagrid-tanzania',
 'geojson-hexagrid-vietnam']

In [4]:
geojson_dic = {k: v for k, v in enumerate(geojson_names)}
geojson_dic


{0: 'geojson-hexagrid-india',
 1: 'geojson-hexagrid-kenya',
 2: 'geojson-hexagrid-nigeria',
 3: 'geojson-hexagrid-tanzania',
 4: 'geojson-hexagrid-vietnam'}

In [5]:
geojson_dic = {k:v for k,v in enumerate(geojson_names)}
geojson_dic


{0: 'geojson-hexagrid-india',
 1: 'geojson-hexagrid-kenya',
 2: 'geojson-hexagrid-nigeria',
 3: 'geojson-hexagrid-tanzania',
 4: 'geojson-hexagrid-vietnam'}

In [6]:
if bool(geojson_dic):
    print("Dictionary is not empty")

Dictionary is not empty


In [7]:
# define the user input

while True:
    if bool(geojson_dic):
        try:
            user_d_spec = int(input(f"Which data do you want to download?\
                                Enter {geojson_dic}"
                                    ))
        except ValueError:
            print("Please enter a number.")
            continue
        if user_d_spec not in range(len(geojson_dic)):
            print(f"Please enter {len(geojson_dic.keys())}")
            continue
        break
    else:
        print("Geojson_dic is empty, restart the kernel, please.")
        break

while True:
    try:
        user_d_format = int(input("Which data format do you want to export?\
                            Enter 0 for json format\
                            or 1 for geojson format"
                                ))
    except ValueError:
        print("Please enter a number.")
        continue
    if user_d_format not in range(0, 2):
        print("Please enter 0 or 1.")
        continue
    break


In [8]:
# define the functions

def convert_geojson_structure(original_dict):
    new_dict = {}
    new_dict["type"] = "FeatureCollection"
    new_dict["features"] = []

    feature_dict = {}
    feature_dict["type"] = "Feature"
    feature_dict["geometry"] = {}
    feature_dict["geometry"]["type"] = "MultiPolygon"

    new_dict["features"].append(feature_dict)

    feature_dict["geometry"]["coordinates"] = original_dict["geometry"]["coordinates"]
    # feature_dict["geometry"]["type"] = "MultiPolygon"
    feature_dict["properties"] = original_dict["properties"]

    return new_dict

def data_gen(feature):

    desired_geojson_structure = convert_geojson_structure(feature)

    parameters = {
        "bpolys": json.dumps(bpolys),  # pass GeoJSON as string.
        "filter": "type: way and (highway in (motorway,\
                                              motorway_link, trunk, trunk_link,\
                                              primary, primary_link, secondary,\
                                              secondary_link, tertiary,\
                                              tertiary_link, unclassified,\
                                              residential, living_street, pedestrian, footway, cycleway, track, path)\
                                  or (highway=service and service=alley))",
        "groupByKey": "source",
        "groupByValues": "maxar",
        "format": "json",
        "time": "2018-01-01/2023-01-01/P1Y",
    }
    headers = {
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    for value in parameters.values():
        assert value != "", "Please provide values for the parameters"

    while True:
        try:
            response = requests.post(url, data=parameters, headers=headers)
            response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

            # print("Response:")
            # print(desired_geojson_structure["features"][0]["properties"]["NAME_EN"])
            # print(json.dumps(response.json(), indent=4))  # Pretty print response

            result = response.json()["groupByResult"]

            for region in result:
                region["groupByObject"][0] = desired_geojson_structure["features"][0]["properties"]["id"]
                # region["groupByObject"].append(desired_geojson_structure["features"][0]["properties"]["ADM0_ISO"])

            return result
        except requests.exceptions.RequestException:
            # Wi-Fi connection error occurred, wait for connection to be restored
            print("Waiting for Wi-Fi connection to be restored...")
            time.sleep(5)  # Wait for 5 seconds
            continue  # Continue to the next iteration of the loop


def connect_feat_data(feature, data):

    rem_value_to_add = None
    ai_value_to_add = None

    for elem in data:
        if elem['groupByObject'][1] == 'remainder':
            rem_value_to_add = find_value(elem, '2023-01-01T00:00:00Z')
        elif elem['groupByObject'][1] == 'source=maxar':
            ai_value_to_add = find_value(elem, '2023-01-01T00:00:00Z')

    roads_total = rem_value_to_add + ai_value_to_add\
        if rem_value_to_add is not None and ai_value_to_add is not None else None
    ai_percentage = (ai_value_to_add / (ai_value_to_add + rem_value_to_add)) * 100\
        if rem_value_to_add is not None and ai_value_to_add is not None else None

    if rem_value_to_add is not None:
        feature['properties']['mm_roads_2023'] = rem_value_to_add

    if ai_value_to_add is not None:
        feature['properties']['ai_roads_2023'] = ai_value_to_add

    if roads_total is not None:
        feature['properties']['roads_total'] = roads_total

    if ai_percentage is not None:
        feature['properties']['ai_percentage'] = ai_percentage

    return feature


def find_value(elem, timestamp):
    for result in elem['result']:
        if result['timestamp'] == timestamp:
            return result['value']
    return None


def return_json_data():

    data = []

    features = bpolys["features"]
    for i, feature in tqdm(enumerate(features), total=len(features)):

        obtained_data = data_gen(feature)
        # print(data)
        data.append(obtained_data)

    return data


def return_geojson_data():

    data = []

    features = bpolys["features"]
    for i, feature in tqdm(enumerate(features), total=len(features)):

        obtained_data = data_gen(feature)
        # print(data)
        feat_with_data = connect_feat_data(feature, obtained_data)
        # print(feat_with_data)
        # print(feature)

        data.append(feat_with_data)

    return data

# export the data


def export_data_as_json(name, data):
    with open(f"downloaded-data\\roads_{name}.json", "w") as file:
        json.dump(data, file, indent=4)


# export_data_as_json(data_dic[user_d_spec], return_json_data)


def export_data_as_geojson(name, data):

    # Define the filename for the GeoJSON file
    filename = f'roads_{name}.geojson'

    # Create a FeatureCollection from the structure
    feature_collection = {
        'type': 'FeatureCollection',
        'features': data
    }

    # Write the FeatureCollection to a GeoJSON file
    with open(f"downloaded-data-geojson\\{filename}", 'w') as file:
        json.dump(feature_collection, file, indent=4)


# export_data_as_geojson(data_dic[user_d_spec], return_geojson_data)


In [9]:
# define the URL

base_url = "https://api.ohsome.org/v1"
endpoint = "/elements/count/groupBy/boundary/groupBy/tag"
url = base_url + endpoint

#### Get the data

In [10]:
# read geojson data

# os.chdir('..')
home_wd = os.getcwd()
# downloaded_data_geojson_dir = os.path.join(home_wd, "downloaded-data-geojson", "")

with open(f"geojson-regions\\geojson-hexagrid-states\\{geojson_dic[user_d_spec]}.geojson", "r") as file:
    bpolys = json.load(file)


In [11]:
# convert_geojson_structure(bpolys["features"][0])

In [12]:
# convert_geojson_structure(bpolys["features"][3])


In [13]:
json_data = []
geojson_data = []

if user_d_format == 0:
    obtained_json_data = return_json_data()
    json_data.extend(obtained_json_data)
    export_data_as_json(geojson_dic[user_d_spec], obtained_json_data)
else:
    obtained_geojson_data = return_geojson_data()
    geojson_data.extend(obtained_geojson_data)
    export_data_as_geojson(geojson_dic[user_d_spec], obtained_geojson_data)


  0%|          | 0/2499 [00:00<?, ?it/s]

Waiting for Wi-Fi connection to be restored...
Waiting for Wi-Fi connection to be restored...
Waiting for Wi-Fi connection to be restored...
Waiting for Wi-Fi connection to be restored...
Waiting for Wi-Fi connection to be restored...


  3%|▎         | 70/2499 [06:47<3:55:37,  5.82s/it]


KeyboardInterrupt: 

In [ ]:
geojson_data


[{'type': 'Feature',
  'properties': {'fid': 422847,
   'id': 4761723,
   'timestamp': '2023-01-01T00:00:00Z',
   'value': 3559.5582118661614,
   'value_per_area': 3.717992957623663e-05,
   'mm_roads_2023': 2.0,
   'ai_roads_2023': 0.0,
   'roads_total': 2.0,
   'ai_percentage': 0.0},
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[103.18194429999998, 22.6255865],
      [103.1481844, 22.5834577],
      [103.08250179999999, 22.5844456],
      [103.05051509999998, 22.6275883],
      [103.08423149999999, 22.669747699999984],
      [103.1499782, 22.6687339],
      [103.18194429999998, 22.6255865]]]]}},
 {'type': 'Feature',
  'properties': {'fid': 422848,
   'id': 4761730,
   'timestamp': '2023-01-01T00:00:00Z',
   'value': 40100.60485263911,
   'value_per_area': 0.00041889571897484745,
   'mm_roads_2023': 2.0,
   'ai_roads_2023': 0.0,
   'roads_total': 2.0,
   'ai_percentage': 0.0},
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[103.1693716, 22.027985699999988],


In [ ]:
import geopandas as gpd

In [ ]:
# gpd.read_file(geojson_data)


In [ ]:
# gpd.GeoDataFrame(geojson_data).set_geometry('geometry')
